This is the main Sensor Kennel application that will 
identify, read and plot any sensor that's plugged in.

On the lower side:  Pin1 (left) Gnd, Pin3 3v3, Pin-3 (from right) SDA, Pin-5 SCL.

In [105]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [44]:
print(i2c.scan())

[57, 60]


In [200]:
%sendtofile --source SI7021_funcs.py
%sendtofile --quiet --source TSL561_funcs.py
%sendtofile --quiet --source device_detection.py
%sendtofile --quiet --binary --source OLED_driver.py

Sent 55 lines (2042 bytes) to SI7021_funcs.py.
Sent 54 lines (1757 bytes) to TSL561_funcs.py.
Sent 29 lines (920 bytes) to device_detection.py.
Sent 3084 bytes in 103 chunks to OLED_driver.py.


In [201]:
%sendtofile main.py

from OLED_driver import i2c, fbuff, oledshow, doublepixels, fatntext

import time
time.sleep(0.5)
from device_detection import IdentifyI2CDevice

devices = IdentifyI2CDevice(i2c)
devname = "nothing"
for dev in devices:
    devname, devdesc = dev.split(" ", 1)
    fbuff.fill(0)
    fbuff.text(devname,0,0,1)
    fbuff.text(devdesc,0,12,1)
    doublepixels()
    oledshow()
    time.sleep(2)
    
if devname == "SI7021":
    from SI7021_funcs import setupSI7021, SI7021humiditytemp
    setupSI7021(i2c)
    while True:
        h, t = SI7021humiditytemp()
        fbuff.fill(0)
        fbuff.text("SI7021", 0, 0, 1)
        fbuff.text("humid:", 0, 23, 1)
        fatntext("%.1f"%h, 50, 16)
        fbuff.text("temp:", 0, 53, 1)
        fatntext("%.1f"%t, 50, 46)
        oledshow()

if devname == "VL53L0X":
    from VL53L0X_funcs import VL53L0Xinit, VL53L0Xdist
    VL53L0Xinit(i2c)
    while True:
        d = VL53L0Xdist()
        fbuff.fill(0)
        fbuff.text("VL53L0X", 0, 0, 1)
        fbuff.text("dist: %.1f"%d, 8, 20, 1)
        oledshow()

if devname == "TSL561":
    from TSL561_funcs import setupTSL561, luminosity, setregtimings
    setupTSL561(i2c)
    setregtimings(True, 1)
    while True:
        br, ir = luminosity()
        fbuff.fill(0)
        fbuff.text("TSL561", 0, 0, 1)
        fbuff.text("broad:", 0, 23, 1)
        fatntext("%d"%br, 50, 16)
        fbuff.text("ir:", 0, 53, 1)
        fatntext("%d"%ir, 50, 46)
        oledshow()

if defname == "TMP102":
    while True:
        r = i2c.readfrom_mem(0x48, 0x00, 2)
        t = r[0] + r[1]/256
        fbuff.fill(0)
        fbuff.text("TMP102", 0, 0, 1)
        fbuff.text("temp:", 0, 23, 1)
        fatntext("%.03f"%t, 30, 40)
        oledshow()


Sent 53 lines (1458 bytes) to main.py.


In [204]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.
